In [1]:
# %%capture
# !pip install auto-gptq
# !pip install optimum
# !pip install bitsandbytes
# # !pip install torch==2.1

In [5]:
# %%capture
# !pip install torchvision==0.15.0

In [ ]:
# %%capture
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [1]:
%%capture
!pip install auto-gptq #allows to run LLM using GPTQ algorithm. Basically brings the memory savings
!pip install optimum #an extension of transformers that provides a set of performance optimzation tools to train and run models with max efficiency.
!pip install bitsandbytes #python wrapper, used with 8-bit optimizers, matmul and 8&4 quantization techniques.
!pip install pynvml
!pip install accelerate
!git clone https://github.com/shaankhosla/NLP_with_LLMs/
%cd "NLP_with_LLMs"

In [2]:
import csv
import random
from datasets import Dataset, DatasetDict
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers
import gpu_utilities, generate_data

In [3]:
#loading the model without unsloth but with smashed version of the model.
model_name_smashed = 'PrunaAI/mistralai-Mistral-7B-Instruct-v0.2-bnb-4bit-smashed'
model_smashed = AutoModelForCausalLM.from_pretrained(model_name_smashed,
                                             device_map = 'auto', #for best use of CPU or GPU.
                                             trust_remote_code=False, #not to run custom files on local machine.
                                             revision="main") #version of the model.

config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.45G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

#### Using tokenizer of the base model on which 'mistralai-Mistral-7B-Instruct-v0.2-bnb-4bit-smashed' is based.

In [4]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
# craft prompt
comment = """Provide a concise summary of the following article, this is latest article from Yahoo, this is the news about stocks and comments from the business leaders.
    Please give sentiments of the stock or crypto currency mentioned in the article, users should be able to tell if that is a good opportunity to invest or should they backout?
    Provide the summary in bullet points."""

prompt=f'''[INST] {comment} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model_smashed.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> [INST] Provide a concise summary of the following article, this is latest article from Yahoo, this is the news about stocks and comments from the business leaders.
    Please give sentiments of the stock or crypto currency mentioned in the article, users should be able to tell if that is a good opportunity to invest or should they backout?
    Provide the summary in bullet points. [/INST] * Article from Yahoo Finance discusses the latest stock market trends and comments from business leaders
* Tesla's stock price surged after Elon Musk announced a 10-for-1 stock split
* Musk also revealed that Tesla would be buying $1.5 billion in Bitcoin, causing a significant increase in Bitcoin's price
* Apple reportedly plans to start production of its new iPhone SE model in India, which could help the company reduce production costs and avoid tariffs
* Microsoft's CEO, Satya Nadella, expressed optimism about the company's growth prospects, particularly in the cloud computing and gaming sectors

In [6]:
gpu_utilities.print_gpu_utilization()

Device 0 : Tesla T4
GPU memory occupied: 5000 MB.


In [26]:
prompt_template = """
    Provide a concise summary of the following article, this is latest article from Yahoo, this is the news about stocks and comments from the business leaders.
    Please give sentiments of the stock or crypto currency mentioned in the article, users should be able to tell if that is a good opportunity to invest or should they backout?
    Provide the summary in bullet points.
"""

question = 'What did Mark Cuban said in the latest news?'

template = lambda question: f'''<s> [INST] {prompt_template}\n {question} [/INST]<s>'''
prompt = template(question)
# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model_smashed.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s>  [INST] 
    Provide a concise summary of the following article, this is latest article from Yahoo, this is the news about stocks and comments from the business leaders.
    Please give sentiments of the stock or crypto currency mentioned in the article, users should be able to tell if that is a good opportunity to invest or should they backout?
    Provide the summary in bullet points.

 What did Mark Cuban said in the latest news? [/INST]<s> Question: What did Mark Cuban say in the latest news and should users invest or back out based on the sentiment towards the mentioned stocks and cryptocurrencies?

Answer:

- Mark Cuban, the billionaire entrepreneur and Dallas Mavericks owner, expressed his bullish outlook on Bitcoin (BTC) and Ethereum (ETH) in a CNBC interview.
- He believes that Bitcoin and Ethereum are here to stay and that they will continue to be a part of the financial system.
- Cuban also mentioned that he owns some Bitcoin and Ethereum, and he encourages others to 

In [8]:
%%time
# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model_smashed.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> [INST] 
    Provide a concise summary of the following article, this is latest article from Yahoo, this is the news about stocks and comments from the business leaders.
    Please give sentiments of the stock or crypto currency mentioned in the article, users should be able to tell if that is a good opportunity to invest or should they backout?
    Provide the summary in bullet points.

 [/INST]<s> Question: Summarize the following Yahoo article about stocks and crypto, provide sentiments, and investment advice.

Answer:

- Title: "Microsoft Earnings Beat Estimates, Boosts Stock; Tesla Slips on Delivery Concerns"
- Microsoft Corporation (MSFT) reported stronger-than-expected earnings for its fiscal first quarter, leading to a 5% increase in its stock price.
- Microsoft's personal computing division, which includes Windows and Surface hardware, saw a 12% revenue growth.
- The company's gaming division, which includes the Xbox, also reported a 21%
CPU times: user 11.8 s, sys: 340 

## Gradient checkpointing

This is a technique that saves strategically selected activations so that only selected selected activation needs to be recomputed for the gradients. This techniques works well because there won't be any need to save all the activations during forward pass to compute the gradient, which could create a big over head.



In [9]:
#Train the model
model_smashed.train()
model_smashed.gradient_checkpointing_enable()
model_smashed = prepare_model_for_kbit_training(model_smashed)

## LoRA

Low-Rank Adaptation (LoRA) is a PEFT method that decomposes a large matrix into two smaller low-rank matrices in the attention layers. This drastically reduces the number of parameters that need to be fine-tuned. [3]

Rank 'r' corresponds to the number of parameters in the adaptation layers -- the more parameters, the better it remembers, and the more complex things it can pick up. Whether this falls into the overfitting regime is debated, but if you have a lot of data covering a variety of types of knowledge and tasks, then you'll need a higher rank for that information to "stick". [4]

'Alpha' is a scaling factor -- it changes how the adaptation layer's weights affect the base model's. Higher alpha means the LoRA layers act more strongly on the base model. [4]

In [10]:
#LoRA config
#why to use fp16 - [2]
config = LoraConfig(
    r=8, #setting rank to 8 means there will be 8 independent rows or columns in the matrix, more the rank, better it will remember.
    lora_alpha=32,
    target_modules=["q_proj"], #target is Q linear model here, if dataset is relatively smalled then one or two linear models are enough or else we will fall into case of overfitting.
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model_smashed = get_peft_model(model_smashed, config)

# trainable parameter count
model_smashed.print_trainable_parameters()

trainable params: 2,097,152 || all params: 7,243,829,248 || trainable%: 0.0290


In [11]:
data = load_dataset('shouray/Latest-News-Articles')

README.md:   0%|          | 0.00/361 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9 [00:00<?, ? examples/s]

In [12]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["example"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

### why padding tokens

Why would you do this? In some models and contexts, it can be beneficial to treat the end of a sequence and padding as the same, especially in models where the distinction between padding and the end of a sequence is not crucial or might interfere with how the model processes the text. For example, in some generative models, having distinct padding and EOS tokens might lead to unwanted behavior during text generation, and using the same token for both can mitigate such issues. [5]

In [13]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token #This means that the same token will be used both to pad shorter sequences and to signify the end of a sequence.
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [15]:
# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "/content/YT_Results",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# configure trainer
trainer = transformers.Trainer(
    model=model_smashed,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
)


# train model
model_smashed.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# renable warnings
model_smashed.config.use_cache = True

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_aut

Epoch,Training Loss,Validation Loss
1,0.567800,2.542387
2,0.567800,2.537058
3,0.553300,2.523864
4,0.522600,2.509232
5,0.494100,2.495176
6,0.470000,2.482280
7,0.450900,2.471992
8,0.435400,2.463906
9,0.424000,2.459227
10,0.416200,2.456910


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

In [17]:
hf_name = 'shouray' # your hf username or org name
model_id = hf_name + "/" + "Mistral-Smashed-Latest-News-Yahoo"

In [18]:
model_smashed.push_to_hub(model_id)
trainer.push_to_hub(model_id)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

events.out.tfevents.1729218611.cd8a2c5e5c2b.260.0:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shouray/YT_Results/commit/9a8f45a6caf2612d25e75bec388d1f1e7e30287c', commit_message='shouray/Mistral-Smashed-Latest-News-Yahoo', commit_description='', oid='9a8f45a6caf2612d25e75bec388d1f1e7e30287c', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
PEFT_MODEL = "shouray/Mistral-Smashed-Latest-News-Yahoo"

In [22]:
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)

In [23]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

adapter_config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

In [25]:
prompt_template = """
    Provide a concise summary of the following article, this is latest article from Yahoo, this is the news about stocks and comments from the business leaders.
    Please give sentiments of the stock or crypto currency mentioned in the article, users should be able to tell if that is a good opportunity to invest or should they backout?
    Provide the summary in bullet points.
"""

question = 'What did Mark Cuban said in the latest news?'

template = lambda question: f'''<s> [INST] {prompt_template}\n {question} [/INST]<s>'''
prompt = template(question)
# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s>  [INST] 
    Provide a concise summary of the following article, this is latest article from Yahoo, this is the news about stocks and comments from the business leaders.
    Please give sentiments of the stock or crypto currency mentioned in the article, users should be able to tell if that is a good opportunity to invest or should they backout?
    Provide the summary in bullet points.

 What did Mark Cuban said in the latest news? [/INST]<s> Question: What did Mark Cuban say in the latest news and should users invest or back out based on the sentiment towards the mentioned stocks and cryptocurrencies?

Answer:

- Mark Cuban, the billionaire entrepreneur and Dallas Mavericks owner, expressed his bullish outlook on Bitcoin (BTC) and Ethereum (ETH) in a CNBC interview.
- He believes that Bitcoin and Ethereum are here to stay and that they will continue to be a part of the financial system.
- Cuban also mentioned that he owns some Bitcoin and Ethereum, and he encourages others to 